In [38]:
ROOT = 'datasets'
DATASET = 'coco'
ANNOTATIONS_PATH = 'annotations/captions_{0}2014.json'
IMAGES_PATH = 'images/{0}2014'

In [5]:
import os

In [39]:
import torchvision.datasets as dset
import torchvision.transforms as transforms

cap = dset.CocoCaptions(root = os.path.join(ROOT, DATASET, IMAGES_PATH.format('train')),
                        annFile = os.path.join(ROOT, DATASET, ANNOTATIONS_PATH.format('train')),
                        transform=transforms.ToTensor())

print('Number of samples: ', len(cap))

loading annotations into memory...
Done (t=0.44s)
creating index...
index created!
Number of samples:  82783
